In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os

In [2]:
games_file = 'Video_Games_Sales_as_at_22_Dec_2016.csv'
games_df = pd.read_csv(games_file)
games_df.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
review_file = 'gamecomm.csv'
review_df = pd.read_csv(review_file, error_bad_lines=False)
review_df.columns = review_df.columns.str.replace(' ', '')
review_df.head()

b'Skipping line 482: expected 8 fields, saw 9\n'


,game_title,editorial_rating,user_rating,game_price,editor,type,date,device
0,Kingpin : Life of Crime sur PC,19,17.5,8-99€,Interplay,+16ans,Août 1999,PC
1,Batman Arkham City sur PC,19,18.3,3-40€,Rocksteady,Animation,24 Novembre 2011,PC
2,Baldur's Gate II : Shadows of Amn sur PC,19,18.6,25-01€,Interplay,+12ans,03 Novembre 2000,PC
3,Splinter Cell sur PC,19,18.1,1-25€,Ubisoft,Infiltration,27 Février 2003,PC
4,The Witcher 3 : Wild Hunt sur PC,19,18.1,21-19€,CDProjektRED,Fantastique,19 Mai 2015,PC


In [4]:
games_cols = ["Name","Publisher", "NA_Sales", "Global_Sales"]
games_transformed= games_df[games_cols].copy()
#games_transformed.set_index("Name", inplace=True)
games_transformed.head()

,Name,Publisher,NA_Sales,Global_Sales
0,Wii Sports,Nintendo,41.36,82.53
1,Super Mario Bros.,Nintendo,29.08,40.24
2,Mario Kart Wii,Nintendo,15.68,35.52
3,Wii Sports Resort,Nintendo,15.61,32.77
4,Pokemon Red/Pokemon Blue,Nintendo,11.27,31.37


In [5]:
reviews_cols = ["game_title", "editorial_rating", "user_rating", "game_price"]
reviews_transformed= review_df[reviews_cols].copy()
reviews_transformed.set_index("game_title", inplace=True)
reviews_transformed.head()

,editorial_rating,user_rating,game_price
game_title,,,
Kingpin : Life of Crime sur PC,19,17.5,8-99€
Batman Arkham City sur PC,19,18.3,3-40€
Baldur's Gate II : Shadows of Amn sur PC,19,18.6,25-01€
Splinter Cell sur PC,19,18.1,1-25€
The Witcher 3 : Wild Hunt sur PC,19,18.1,21-19€


In [6]:
connection_string = "postgres:Illinifootball25@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{connection_string}')
engine.table_names()

['games', 'reviews']

In [7]:
games_transformed.rename(columns={'Name':'game_title', 'Publisher':'publisher', 'NA_Saless':'na_sales', 'Global_Sales':'global_sales'}, inplace=True)
games_transformed.head()

,game_title,publisher,NA_Sales,global_sales
0,Wii Sports,Nintendo,41.36,82.53
1,Super Mario Bros.,Nintendo,29.08,40.24
2,Mario Kart Wii,Nintendo,15.68,35.52
3,Wii Sports Resort,Nintendo,15.61,32.77
4,Pokemon Red/Pokemon Blue,Nintendo,11.27,31.37


In [8]:
reviews_transformed.rename(columns={'game_title': 'title', 'editorial_rating': 'ed_rating', 'user_rating': 'use_rating', 'game_price': 'price'})
reviews_transformed.head()

,editorial_rating,user_rating,game_price
game_title,,,
Kingpin : Life of Crime sur PC,19,17.5,8-99€
Batman Arkham City sur PC,19,18.3,3-40€
Baldur's Gate II : Shadows of Amn sur PC,19,18.6,25-01€
Splinter Cell sur PC,19,18.1,1-25€
The Witcher 3 : Wild Hunt sur PC,19,18.1,21-19€


In [9]:
 games_transformed.to_sql(name='games', con=engine, if_exists='append', index=True)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "NA_Sales" of relation "games" does not exist
LINE 1: INSERT INTO games (index, game_title, publisher, "NA_Sales",...
                                                         ^
 [SQL: 'INSERT INTO games (index, game_title, publisher, "NA_Sales", global_sales) VALUES (%(index)s, %(game_title)s, %(publisher)s, %(NA_Sales)s, %(global_sales)s)'] [parameters: ({'index': 0, 'game_title': 'Wii Sports', 'publisher': 'Nintendo', 'NA_Sales': 41.36, 'global_sales': 82.53}, {'index': 1, 'game_title': 'Super Mario Bros.', 'publisher': 'Nintendo', 'NA_Sales': 29.08, 'global_sales': 40.24}, {'index': 2, 'game_title': 'Mario Kart Wii', 'publisher': 'Nintendo', 'NA_Sales': 15.68, 'global_sales': 35.52}, {'index': 3, 'game_title': 'Wii Sports Resort', 'publisher': 'Nintendo', 'NA_Sales': 15.61, 'global_sales': 32.77}, {'index': 4, 'game_title': 'Pokemon Red/Pokemon Blue', 'publisher': 'Nintendo', 'NA_Sales': 11.27, 'global_sales': 31.37}, {'inde

In [ ]:
 reviews_transformed.to_sql(name='reviews', con=engine, if_exists='append', index=True)